In [ ]:
# transform

# Dataset

def main(cfg):
    # device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # transform
    transform = get_transform(cfg.hparam.resize, cfg.hparam.mean, cfg.hparam.std)

    # dataset
    dataset = ImageFolder(root=cfg.hparam.test_dir, transform=transform)

    # test loader
    create_loader(dataset, cfg.hparam.batch_size,False, is_train=False ,shuffle=False)

    # model
    num_classes = 2
    model = get_model(cfg.models.resnet50, device)
    in_features = model.fc.in_features
    model.fc = nn.Linear(in_features, num_classes)
    

    # best_model
    best_model_path = cfg.model.checkpoint
    model.eval()

    # inference
    with torch.no_grad():


    # flaten

    # subplot



